<a href="https://colab.research.google.com/github/june-oh/cseg109/blob/main/LAB5_Seq2Seq_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Moo/korean-parallel-corpora",split='train')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[0]

{'ko': '개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"',
 'en': 'Much of personal computing is about "can you top this?"'}

In [ ]:
len(dataset['ko'])

96215

In [ ]:
import re

def preprocess_sentence(sentence,ko=False):
    if ko:
      sentence = sentence.strip()
      sentence = re.sub(r"([?.!,])", r" \1 ", sentence) #특수문자
      sentence = re.sub(r'[" "]+', " ", sentence) #여러공백
      sentence = re.sub(r"[^0-9a-zA-Z가-힣?.!,]+", " ", sentence) #한글 영어 숫자 제외 제거
      sentence = sentence.strip()

    else:
      sentence = sentence.lower().strip()
      sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
      sentence = re.sub(r'[" "]+', " ", sentence)
      sentence = re.sub(r"[^0-9a-zA-Z?.!,]+", " ", sentence) #영어 숫자 제외 제거

      sentence = sentence.strip()

    sentence = '<bos> ' + sentence + ' <eos>'

    return sentence

In [ ]:
ko_data = [preprocess_sentence(s,ko=True) for s in dataset['ko']]

In [ ]:
ko_data[:10]

['<bos> 개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ? <eos>',
 '<bos> 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다 . <eos>',
 '<bos> 그러나 이것은 또한 책상도 필요로 하지 않는다 . <eos>',
 '<bos> 79 . 95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다 . <eos>',
 '<bos> 정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을 밝혔으며 , 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다 . <eos>',
 '<bos> 이 지역에 있는 미국 선박과 상업용 선박들에 대한 알카에다의 테러 시도 중 여러 건이 실패했다는 것을 알게 된 후에 , 전문가들은 테러 조직이 여전히 세계 경제에 타격을 입히려 한다고 경고하고 있으며 , 동남 아시아에 있는 세계 경제의 주요 통로가 위험에 처해 있다고 그들은 생각하고 있다 . <eos>',
 '<bos> 국립 과학 학회가 발표한 새 보고서에따르면 , 복잡한 임무를 수행해야 하는 군인들이나 보다 오랜 시간 동안 경계를 늦추지 않고 있기 위해 도움이 필요한 군인들에게 카페인이 반응 시간을 증가시키고 임무 수행 능력을 향상시키는데 도움이 된다고 한다 . <eos>',
 '<bos> 이 보고서에따르면 , 특히 , 군사 작전에서 생사가 걸린 상황이 될 수도 있는 반응 속도와 시각 및 청각의 경계 상태를 유지시키기 위해 카페인이 사용될 수도 있다 . 고 한다 . <eos>',
 '<bos> 결정적인 순간에 그들의 능력을 증가시켜 줄 그 무엇이 매우 중요합니다 . <eos>',
 '<bos> 연구가들이 이미 커피 대체품으로서 음식 대용 과자나 껌에 카페인을 첨가하는 방법을 연구하고 있다고 Archibald는 말했다 

In [ ]:
en_data =  [preprocess_sentence(s,ko=False) for s in dataset['en']]

In [ ]:
en_data[:10]

['<bos> much of personal computing is about can you top this ? <eos>',
 '<bos> so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable , wireless mouse . <eos>',
 '<bos> like all optical mice , but it also doesn t need a desk . <eos>',
 '<bos> uses gyroscopic sensors to control the cursor movement as you move your wrist , arm , whatever through the air . <eos>',
 '<bos> intelligence officials have revealed a spate of foiled plots on ships in southeast asia and are warning that a narrow stretch of water carrying almost one third of the world s maritime trade is vulnerable to a terror attack . <eos>',
 '<bos> after learning of several foiled al qaeda attempts on u . s . and commercial ships in the area , experts are warning that the terror network still wants to cripple the global economy , the world s economic jugular vein in southeast asia is at risk . <eos>',
 '<bos> caffeine can help increase reaction time and improve performance for m

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict

# 단어 사전 생성 함수
def build_vocab(data):
    vocab = {}
    # 초기 토큰 설정
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1
    vocab["<bos>"] = 2
    vocab["<eos>"] = 3

    # 단어 인덱스 시작점 설정
    idx = 4

    # 데이터에서 단어 추출 및 인덱스 할당
    for sentence in data:
        for word in sentence.split():
            if word not in vocab:
                vocab[word] = idx
                idx += 1

    return vocab


# 단어 사전 생성
ko2idx = build_vocab(ko_data)
en2idx = build_vocab(en_data)
idx2ko = {val:key for key,val in ko2idx.items()}
idx2en = {val:key for key,val in en2idx.items()}

# Custom Dataset 클래스 정의
class TranslationDataset(Dataset):
    def __init__(self, src_data, trg_data, src_vocab, trg_vocab):
        self.src_data = [[src_vocab[word] for word in sentence.split()] for sentence in src_data]
        self.trg_data = [[trg_vocab[word] for word in sentence.split()] for sentence in trg_data]

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return torch.tensor(self.src_data[idx]), torch.tensor(self.trg_data[idx])

# 패딩 함수 정의
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    trg_padded = pad_sequence(trg_batch, batch_first=True, padding_value=0)
    return src_padded, trg_padded

# 데이터셋 생성
dataset = TranslationDataset(ko_data, en_data, ko2idx, en2idx)

# DataLoader 생성
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)


In [ ]:
next(iter(dataloader))[1].shape

torch.Size([32, 51])

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout,batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout,batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden

In [ ]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg):
        trg_len = trg.shape[1]
        batch_size = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        _, hidden = self.encoder(src)
        hidden = hidden[:,:src.size(0),:]

        input = trg[:,0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t] = output
            # print(output.shape)
            input = output.argmax(1)
            # print(input.shape)

        return outputs

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

INPUT_DIM = len(ko2idx)
OUTPUT_DIM = len(en2idx)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=ko2idx["<pad>"])

In [ ]:
from tqdm.auto import tqdm

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for i, (src, trg) in tqdm(enumerate(iterator),leave=False,total=len(iterator)):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# 학습 실행
N_EPOCHS = 1

for epoch in tqdm(range(N_EPOCHS)):
    train_loss = train(model, dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3007 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_len=50):
    model.eval()

    # 한국어 문장 토큰화 및 정수 인덱스 변환
    if isinstance(sentence, str):
        tokens = [token for token in sentence.split()]
    else:
        tokens = [token for token in sentence]
    tokens = [src_vocab.get(token, src_vocab["<unk>"]) for token in tokens]

    # 모델의 입력 형태로 변환
    src_indexes = [src_vocab["<bos>"]] + tokens + [src_vocab["<eos>"]]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # Encoder를 통한 추론
    with torch.no_grad():
        _, hidden = model.encoder(src_tensor)

    # Decoder 초기 입력 설정
    trg_indexes = [en2idx["<bos>"]]

    # Decoder를 통한 추론
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        # 문장 종료 조건 확인
        if pred_token == en2idx["<eos>"]:
            break

    # 정수 인덱스를 단어로 변환
    trg_tokens = [idx2en.get(idx, "<unk>") for idx in trg_indexes]

    # 결과 문장 반환
    return trg_tokens[1:]  # <bos> 토큰 제거

# 예시 문장 번역
sentence = "한국어를 영어로 번역하는 모델입니다."
translation = translate_sentence(sentence, ko2idx, idx2en, model, device)
print(" ".join(translation))


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [ ]:
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim

    def forward(self, hidden, encoder_outputs):
        # hidden: [num_layer,batch size, dec hid dim]
        # encoder_outputs: [batch size, src len, enc hid dim]
        # print(encoder_outputs.shape,hidden.shape)
        # 마지막 레이어의 hidden state만 사용
        hidden = hidden[-1].unsqueeze(2)  # hidden: [batch size, dec hid dim, 1]
        #print(encoder_outputs.shape,hidden.shape)
        attention = torch.bmm(encoder_outputs, hidden).squeeze(2)
        # usinf F.softmax
        return F.softmax(attention, dim=1)


In [ ]:
class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim,n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim + emb_dim, dec_hid_dim, n_layers,batch_first=True)
        self.fc_out = nn.Linear(enc_hid_dim + dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))

        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)

        weighted = torch.bmm(a, encoder_outputs)

        rnn_input = torch.cat((embedded, weighted), dim=2)

        # hidden 텐서의 차원 조정
        # hidden: [num_layers, batch size, dec hid dim]
        print("decoder hidden shape",hidden.shape)
        print("decoder input shape",rnn_input.shape)
        output, hidden = self.rnn(rnn_input, hidden)

        output = output.squeeze(1)
        weighted = weighted.squeeze(1)

        prediction = self.fc_out(torch.cat((output, weighted), dim=1))

        return prediction, hidden.squeeze(0)


In [ ]:
class Seq2SeqAttn(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[1]
        batch_size = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs


In [ ]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
attention = Attention(HID_DIM)
dec = DecoderAttn(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, HID_DIM, N_LAYERS,DEC_DROPOUT, attention)

# Seq2Seq 모델 초기화
model_Attn = Seq2SeqAttn(enc, dec, device).to(device)

In [ ]:
from tqdm.auto import tqdm

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for i, (src, trg) in tqdm(enumerate(iterator),leave=False,total=len(iterator)):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# 학습 실행
N_EPOCHS = 1

for epoch in tqdm(range(N_EPOCHS)):
    train_loss = train(model_Attn, dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3007 [00:00<?, ?it/s]

torch.Size([32, 48, 512]) torch.Size([2, 32, 512])
torch.Size([32, 48, 512]) torch.Size([32, 512, 1])
decoder hidden shape torch.Size([2, 32, 512])
decoder input shape torch.Size([32, 1, 768])
torch.Size([32, 48, 512]) torch.Size([2, 32, 512])
torch.Size([32, 48, 512]) torch.Size([32, 512, 1])
decoder hidden shape torch.Size([2, 32, 512])
decoder input shape torch.Size([32, 1, 768])
torch.Size([32, 48, 512]) torch.Size([2, 32, 512])
torch.Size([32, 48, 512]) torch.Size([32, 512, 1])
decoder hidden shape torch.Size([2, 32, 512])
decoder input shape torch.Size([32, 1, 768])
torch.Size([32, 48, 512]) torch.Size([2, 32, 512])
torch.Size([32, 48, 512]) torch.Size([32, 512, 1])
decoder hidden shape torch.Size([2, 32, 512])
decoder input shape torch.Size([32, 1, 768])
torch.Size([32, 48, 512]) torch.Size([2, 32, 512])
torch.Size([32, 48, 512]) torch.Size([32, 512, 1])
decoder hidden shape torch.Size([2, 32, 512])
decoder input shape torch.Size([32, 1, 768])
torch.Size([32, 48, 512]) torch.Siz

KeyboardInterrupt: ignored